In [ ]:
sm = snakemake

Aim: make overviews of whole sites (=panorama) to be used for fig1


In [ ]:
import pandas as pd
import numpy as np
import plotnine as gg
import seaborn as sns
import matplotlib.pyplot as plt
import os 
import re
import seaborn as sns

import pathlib

In [ ]:
import pycytools as pct
import pycytools.library
import pycytools.plots

In [ ]:
import spherpro.bro as spb
import spherpro.datastore as spd
import spherpro.library as spl
import spherpro.configuration as conf
import spherpro.db as db
import sqlalchemy as sa

In [ ]:
import scipy.ndimage.filters as filters
from scipy.ndimage.filters import gaussian_filter
from skimage.color import label2rgb

In [ ]:
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
import colorcet

In [ ]:
from src.config import Conf as C
from src.variables import Vars as V

In [ ]:
# move this into a global 'prepare folders function'
C.fn_config = sm.input.fn_config
C.fol_plts = pathlib.Path('results/figures/')
C.fol_plts.mkdir(exist_ok=True)

In [ ]:
bro = spb.get_bro(C.fn_config)

In [ ]:
def _read_and_concat(fol_ome, suffix, idname):
    ac_names = [f for f in os.listdir(fol_ome) if os.path.isdir(os.path.join(fol_ome, f))]
    dat = pd.concat([pd.read_csv(os.path.join(fol_ome, a, a+suffix)) for a in ac_names],
                           keys=ac_names, names=[V.COL_ACSESSION, V.VAL_TEMP])
    dat = dat.reset_index(V.VAL_TEMP, drop=True)
    dat = dat.reset_index()
    dat = dat.rename(columns={V.COL_MCD_ID: V.COL_ACID})
    return dat 

def read_acmeta(fol_ome):
    dat_acmeta = _read_and_concat(fol_ome, V.SUFFIX_ACMETA, V.COL_ACID)
    return dat_acmeta

In [ ]:
def get_channel_images(acs, channels):
    imgs = acs.map(lambda x: x.get_img_by_metal(channels))
    return imgs
    


In [ ]:
def get_imc_acimg(x):
    return bro.io.imcimg.get_imc_acquisition(x[db.slideacs.slideac_name.key], x[db.acquisitions.acquisition_mcd_acid.key])
    


In [ ]:
def get_img(site_id, channel, dat_acmeta_c, dat_site_c, filterfkt=None, norm=None, perc=99.999):
    pdat =(dat_site_c
           .query(f'{db.sites.site_id.key} == {site_id}')
           .merge(dat_acmeta_c, left_on=(db.slideacs.slideac_name.key, db.acquisitions.acquisition_mcd_acid.key),
                  right_on=(V.COL_ACSESSION, V.COL_ACID)))
    img_dim = (np.max(pdat[V.COL_X_START]+pdat[V.COL_MAXX]), np.max(pdat[V.COL_Y_START]+pdat[V.COL_MAXY]))
    img_big = np.zeros(img_dim)
    img_big[:] = np.NAN
    if ~(V.COL_IMCAC in pdat.columns):
        pdat[V.COL_IMCAC] = (pdat.apply(get_imc_acimg, axis=1)
        )       
    
    pdat = pdat.sort_values(V.COL_ACID, ascending=False)
    for x,y,img in zip(pdat[V.COL_X_START], pdat[V.COL_Y_START], get_channel_images(pdat[V.COL_IMCAC], channel)):
        img_big[x:(x+img.shape[1]), y:(y+img.shape[0])] = (np.fliplr(img.T))
        #img_big[x:(x+img.shape[1]), y:(y+img.shape[0])] = ((img.T))

    return img_big




In [ ]:
fols_ome = bro.data.conf[conf.CPOUTPUT][conf.IMAGES_CSV][conf.IMAGE_OME_FOLDER_DIRS]

In [ ]:
fols_ome

In [ ]:
dat_acmeta = pd.concat([read_acmeta(f) for f in fols_ome])

1) get all acquisition from a site

2) get the imc acquisitions

In [ ]:
dat_ac = bro.doquery(bro.session.query(db.sites, db.acquisitions, db.slideacs)
           .join(db.acquisitions)
            .join(db.slideacs)
                )
dat_ac = dat_ac.loc[:, ~dat_ac.columns.duplicated()]

In [ ]:
dat_acmeta

In [ ]:
site_id =21

In [ ]:
dat_acmeta[V.COL_X_START] = dat_acmeta.groupby([V.COL_ACSESSION])[V.COL_XSLIDE_START].transform(lambda x: (x-np.min(x))).astype(int)
dat_acmeta[V.COL_Y_START] = dat_acmeta.groupby([V.COL_ACSESSION])[V.COL_YSLIDE_END].transform(lambda x: (x-np.min(x))).astype(int)


In [ ]:
img = get_img(site_id, 'Ir193', dat_acmeta, dat_ac)

In [ ]:

def gauss_75(img):
    img = gaussian_filter(img, 0.75)
    return img

In [ ]:
def process_img(img):
    img = pct.plots._preproc_img_stack([img],sigma=0.75)[0]
    return img

In [ ]:
img_p = process_img(img)

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(img_p)

After looking through these image, I decided for site 21 - this is a site that has only 1 acquisition and looks really nice

In [ ]:

dat_cursite = bro.doquery(bro.session.query(db.images, db.acquisitions)
    .join(db.acquisitions)
    .join(db.sites)
    #.join(db.valid_images)
    .filter(db.sites.site_id == site_id))

dat_cursite = dat_cursite.loc[:, ~dat_cursite.columns.duplicated()]

dat_cursite[db.acquisitions.acquisition_id.key].unique()

In [ ]:
img = get_img(site_id, 'Ir193', dat_acmeta, dat_ac)

In [ ]:
import scipy.ndimage.filters as filters
from scipy.ndimage.filters import gaussian_filter
def gauss_75(img):
    img = gaussian_filter(img, 0.75)
    return img

#%%

def process_img(img):
    img = pct.plots._preproc_img_stack([img],sigma=0.75)[0]
    return img

#%%

img_p = process_img(img)

#%%

plt.figure(figsize=(5,5))
plt.imshow(img_p)

In [ ]:
acquisition_id = dat_cursite[db.acquisitions.acquisition_id.key].unique()[0]

In [ ]:
imgs = list(dat_cursite[db.images.image_id.key])

In [ ]:
imgs

In [ ]:
for i in imgs:
    try:
        fig = bro.plots.debarcoededcells.plot_debarcoded_cells(i, colorbar=True)
        plt.show()
    except:
        pass

In [ ]:
# Select an example image
cur_img=404

In [ ]:
d = bro.plots.heatmask.get_heatmask_data({db.objects.object_type.key: 'cell',
                                                db.ref_planes.channel_name.key: 'object',
                                                db.stacks.stack_name.key: 'ObjectStack', db.measurement_names.measurement_name.key: 'barcode'},
                                                image_ids=imgs,
                                        valid_objects=True,
                                        valid_images=False)

In [ ]:
img = bro.plots.heatmask.assemble_heatmap_image(d)

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
unicols=d[db.object_measurements.value.key].nunique()

In [ ]:
base_colormap=colorcet.glasbey

In [ ]:
color_invalid='#F8F8F8'

In [ ]:
cmap = [color_invalid] + base_colormap

In [ ]:
mymap = mcolors.LinearSegmentedColormap.from_list('mymap', cmap, N=unicols+1)

In [ ]:
mymap.set_bad(color_invalid)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(15,15))
fig = bro.plots.heatmask.do_heatplot(img, cmap=mymap, colorbar=False, cmap_mask_alpha=0.3,ax=ax)
fig.axes.axis('off')

Plot segmentation

In [ ]:
d = bro.plots.heatmask.get_heatmask_data({db.objects.object_type.key: 'cell',
                                                db.ref_planes.channel_name.key: 'Ir193',
                                                db.stacks.stack_name.key: 'FullStackFiltered',
                                          db.measurement_names.measurement_name.key: 'MeanIntensityComp'},
                                                image_ids=imgs,
                                         valid_objects=True,
                                        valid_images=False)

In [ ]:
import random

In [ ]:
random.seed(123213)
d['obj'] = d[db.objects.object_number.key].sample(frac=1,).values
d['img'] = d[db.images.image_id.key]

In [ ]:
img = bro.plots.heatmask.assemble_heatmap_image(d,value_var='obj')

In [ ]:
col = [c for c in colorcet.glasbey]

random.shuffle(col)

In [ ]:
cmap_glasbey  = mcolors.LinearSegmentedColormap.from_list('glasbey', col)
cmap_glasbey.set_bad(color_invalid)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(15,15))
fig = bro.plots.heatmask.do_heatplot(img, cmap=cmap_glasbey, colorbar=False, cmap_mask_alpha=1.5,ax=ax)
fig.axes.axis('off')

In [ ]:
img = bro.plots.heatmask.assemble_heatmap_image(d.query(f'{V.COL_IMGID}=={cur_img}'),value_var='obj')
fig,ax = plt.subplots(1,1, figsize=(15,15))
fig = bro.plots.heatmask.do_heatplot(img, cmap=cmap_glasbey, colorbar=False, cmap_mask_alpha=1.5,ax=ax)
fig.axes.axis('off')

Illustrate the sphere segmentation based on this sphere

- get label mask

In [ ]:
d = bro.plots.heatmask.get_heatmask_data({db.objects.object_type.key: 'cell',
                                                db.ref_planes.channel_name.key: 'is-sphere',
                                                db.stacks.stack_name.key: 'BinStack',
                                          db.measurement_names.measurement_name.key: 'MeanIntensity'},
                                                image_ids=[cur_img],
                                         valid_objects=False,
                                        valid_images=False)
timg = bro.plots.heatmask.assemble_heatmap_image(d)
timg.data[np.isnan(timg.data)] = 0 
img_label_sphere =  np.ma.array((gaussian_filter(timg.data, 10) >0.2).astype(int), mask = gaussian_filter(timg.mask, 1) > 0.01)
bro.plots.heatmask.do_heatplot(img_label_sphere)

- Get Iridium channel

In [ ]:
img_dna = bro.io.stackimg.get_planeimg(cur_img, bro.helpers.dbhelp.get_plane_id('FullStackComp', 'Ir193'))

In [ ]:
bro.plots.heatmask.do_heatplot(np.sqrt(img_dna))

- overlay

In [ ]:
img_lab = np.array(img_label_sphere.data)
img_lab[img_label_sphere.mask == True] = -1
image_label_overlay = label2rgb(img_lab, image=process_img(img_dna)*2, bg_label=-1,bg_color=(0,0,0))
plt.imshow(image_label_overlay)
plt.axis('off')

In [ ]:
timg = bro.plots.heatmask.assemble_heatmap_image(d, value_var=db.objects.object_id.key)
timg.data[np.isnan(timg.data)] = 0 

image_label_overlay = label2rgb(timg.data, image=process_img(img_dna)*2, bg_label=0,bg_color=(0,0,0))
fig = plt.figure()
plt.imshow(image_label_overlay)
plt.axis('off')
fig.set_dpi(200)

Plot markers

In [ ]:
q=(bro.session.query(db.slideacs)
           .join(db.sites)
       .join(db.acquisitions)
       .join(db.images)
       .filter(db.images.image_id == cur_img)
  )
bro.doquery(q)

In [ ]:
imc_img_single = bro.io.imcimg.get_imcimg(cur_img)
imc_img = imc_img_single.original_imcac

In [ ]:
def get_image_slice(row):
    posx = int(row[db.images.image_pos_x.key])
    posy = int(row[db.images.image_pos_y.key])
    w = int(row[db.images.image_shape_w.key])
    h = int(row[db.images.image_shape_h.key])
    cur_slice = np.s_[posx:(h+posx), posy:(w+posy),:]
    return cur_slice

meta = bro.io.imcimg._get_imgmeta(cur_img)
cur_slice = get_image_slice(meta)

In [ ]:
def do_posterplot(imc_img, chans, alphas, basename, cur_slice):
    img = pct.plots.get_7_color_img(imc_img,
                               metals=chans,
                               alphas=alphas, sigma=0.5,
                                   saturation=sat)

    fig, ax =plt.subplots()
    plt.imshow(img)
    scalebar = ScaleBar(0.000001, location=4,color='white',pad=0.1,
                            frameon=False,
                            font_properties={'size': 10, 'family':'normal'}) # 1 pixel = 0.2 meter
    plt.axis('off')
    ax.add_artist(scalebar)
    #plt.savefig(os.path.join(fol_output, basename+'_overview.png'), dpi=300, bbox_inches='tight', pad_inches=0)


    fig, ax =plt.subplots()
    plt.imshow(img[cur_slice])
    scalebar = ScaleBar(0.000001, location=4,color='white',pad=0.1,
                            frameon=False,
                            font_properties={'size': 10, 'family':'normal'}) # 1 pixel = 0.2 meter
    plt.axis('off')
    ax.add_artist(scalebar)
    #plt.savefig(os.path.join(fol_output, basename+'_zoom.png'), dpi=300, bbox_inches='tight', pad_inches=0)
    #|plt.ion()

In [ ]:
vis_settings = [
    [('Yb171', .5), #pS6 
    ('Er166', .5), # Carbonic anhydrase
     
     ('Ir193',0.5),
     (0, 0.5),
    ('Er168', 0.7), #Ki67,
     (0,0.5),
    ('Yb172', 0.5) # # 
    ],
    [('Nd148', .5), #rb 
    ('Yb176', .5), # Survivin
     ('Tb159', 0.3), #GAPDH
     
     (0, 0.5),
    ('Gd158',0.5),#pr
     (0,0.5),
    ('Eu153', 0.5) # # 
    ],
    
    [('Yb174', .7), #actin 
    ('Gd160', .5), # CD44
     ('La139', 0.7), #trimet
     
     (0, 0.5),
    ('Gd156',0.75),#Cycline B1
     (0,0.5),
    ('Sm152', 0.5) # AMPK
    ],
]
basenames = ['img1', 'img2', 'img3']

In [ ]:

sat=1.5


for i, basename in enumerate(basenames):
    chans, alphas = zip(*vis_settings[i]) 
    basename = basenames[i]
    do_posterplot(imc_img, chans, alphas, basename, cur_slice)



Get the slice information for all images

In [ ]:
COL_SLICE = 'image_slice'

In [ ]:
dat_image = bro.doquery(bro.session.query(db.images))
dat_image[COL_SLICE] = dat_image.apply(get_image_slice, axis=1)

Plot Barcode Scheme

In [ ]:
bro.data._read_barcode_key()

In [ ]:
tab_bc =bro.processing.debarcode._get_barcode_key()

In [ ]:
dat_cond = bro.doquery(bro.session.query(db.conditions))
dat_cond = dat_cond

In [ ]:
COL_WELL = db.conditions.well_name.key
COL_METAL = db.ref_planes.channel_name.key
COL_ISPRESENT = 'ispresent'

In [ ]:
bc_scheme = tab_bc
bc_scheme.columns.name = COL_METAL
bc_scheme = bc_scheme.stack()
bc_scheme.name = COL_ISPRESENT

In [ ]:
bc_scheme = (bc_scheme.to_frame()
             .join(dat_cond.set_index(db.conditions.condition_id.key, drop=False)[COL_WELL]
                   .drop_duplicates(), how='inner')
             .set_index(COL_WELL,append=True)
            )

In [ ]:
dat_pannel = bro.data.pannel

In [ ]:

p = 99.9


well_dict = {'%s%02d' % (r, c): (i, j) for (i,r) in enumerate('BCDEFG') for (j,c) in enumerate(range(2, 12))}


#site = dat_image.index.get_level_values('site').unique()[0]
#bc_metals = dat_pannel[dat_pannel['barcoding'] ==1 ]['metal']
for metal, datmetal in bc_scheme.groupby(level=COL_METAL):
    if metal != 'Pd102':
        continue
    fig, ax = plt.subplots(nrows=6, ncols=10, subplot_kw={'xticks': [], 'yticks': []},squeeze=False)
    fig.patch.set_facecolor('black')
    for well, datwell in datmetal.groupby(COL_WELL):
        i,j = well_dict[well]
        curax = ax[i,j]
        col =[float(datwell.values[0]==0) for it in range(3)]
        curax.scatter([1],[1],color=col, s=50, marker='o')
        #curax.set_axis_bgcolor(col)
    plt.suptitle(metal)

In [ ]:
bc_scheme_w = bc_scheme.reset_index(level=COL_WELL,drop=False)
bc_scheme_w['row'] = bc_scheme_w[COL_WELL].map(lambda x: well_dict[x][1])+2
bc_scheme_w['col'] = bc_scheme_w[COL_WELL].map(lambda x: well_dict[x][0])+2
bc_scheme_w['color']  = bc_scheme_w['ispresent'].map(lambda x: [min(float(x==0)+0.3,1) for it in range(3)])
                                                
bc_metals = bc_scheme.index.get_level_values(COL_METAL).unique()

fig, allax = plt.subplots(nrows=2, ncols=4)
for ax, (metal, datmetal) in zip(allax.flatten(), bc_scheme_w.groupby(level=COL_METAL)):   
    ax.scatter(datmetal['row']-1.5, datmetal['col']-1.5,color=datmetal['color'], s=95, edgecolors='black')
    ax.set_yticks(np.arange(7), minor=False)
    ax.yaxis.grid(True, which='major')
    ax.set_xticks(np.arange(11), minor=False)
    ax.xaxis.grid(True, which='major')
    #ax.set_axis_bgcolor('white')
    ax.set_aspect(1)
    ax.set_title(metal)
    ax.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.tick_params(
        axis='y',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.get_yaxis().set_visible(False)
plt.tight_layout()

In [ ]:

color_dict = {'Pd102':(1,0,0), 'Pd104':(0,1,0),'Pd105':(0,0,1),'Pd106':(0,1,1), 'Pd108':(1,0,1),'Pd110':(1,1,0)}
def get_col(p, w, color_dict, default=None):
    if p & (w in color_dict.keys()):
        return color_dict[w]
    else:
        if default is None:
            return (0,0,0)
        else:
            return default

In [ ]:

bc_scheme_w['color_pd']  = [np.array(get_col(p,w, color_dict))*0.333 for p, w in  zip(bc_scheme_w['ispresent'], bc_scheme.index.get_level_values(COL_METAL))]

In [ ]:

datmetal =bc_scheme_w.groupby([COL_WELL, 'row','col'])['color_pd'].apply(lambda x: np.sum(x.T)).reset_index(drop=False)

In [ ]:

datmetal

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=1)

ax.scatter(datmetal['row']-1.5, datmetal['col']-1.5,color=datmetal['color_pd'], s=1537, edgecolors='black')
ax.set_yticks(np.arange(7), minor=False)
ax.yaxis.grid(True, which='major')
ax.set_xticks(np.arange(11), minor=False)
ax.xaxis.grid(True, which='major')
#ax.set_axis_bgcolor('white')
ax.set_aspect(1)
plt.axis('off')
plt.xlim([0,10])
plt.ylim([6,0]) 
ax.get_yaxis().set_visible(False)
plt.tight_layout()

In [ ]:
## Plot color image in plate layout
p = 99.9

sat = 2
img = pct.plots.get_7_color_img(imc_img, sigma=0.7,
             metals=['Yb174', # Pan Cadherin
                     'Er167', #GFP
                     'Eu153', #pHistone
                     'Ir191', #Iridium
                     'Er168', #Ki67
                     'Nd145', #pMAPKAPK
                     'Yb172'],# Cleaved Caspase
            alphas=[0.2, #r
                    0.3, #g
                    0.3, #b
                    0.3, #white
                    0.3, # cyan
                    0., #mangenta
                   0.3], saturation=sat)#yellow

In [ ]:
pdat = (dat_image
        
        .query(f'{db.images.acquisition_id.key}=={acquisition_id}')
        .merge(dat_cond.reset_index(drop=True))
       )

In [ ]:
pdat = (dat_image
        .query(f'{db.images.acquisition_id.key}=={acquisition_id}')
    .merge(dat_cond))

In [ ]:
fig, ax = plt.subplots(nrows=6, ncols=10, subplot_kw={'xticks': [], 'yticks': []},squeeze=True)
fig.patch.set_facecolor('black')

for idx, row in pdat.iterrows():    
    wellid = row[COL_WELL]
    i,j = well_dict[wellid]
    curax = ax[i,j]

    curslice = row[COL_SLICE]
    pimg = img[curslice]
    curax.imshow(pimg)
    curax.set_aspect(1)
    curax.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')
    curax.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')
    scalebar = ScaleBar(0.000001, location=4, color='white',pad=0.1,
                        frameon=False,
                        font_properties={'size': 5, 'family':'normal'}) # 1 pixel = 0.2 meter
    #scalebar.set_units = 'um'
    curax.add_artist(scalebar)
#plt.suptitle(met)
for curax in ax.flatten():
    curax.set_facecolor('black')

In [ ]:
sigma=2
norm_perc=99.9
outlierthresh =30
sat=5
img = pct.plots.get_7_color_img(imc_img, metals=['Pd102', 'Pd104','Pd105', 'Ir191','Pd106', 'Pd108','Pd110'],
                           alphas=[0.33,0.33,0.33,0.01,0.33,0.33,0.33], saturation=sat,
                                           norm_perc=norm_perc, sigma=sigma,
                 outlierthresh=outlierthresh)

In [ ]:
plt.imshow(img)

In [ ]:
imc_img

In [ ]:
fig, ax = plt.subplots(nrows=6, ncols=10, subplot_kw={'xticks': [], 'yticks': []},squeeze=True)
fig.patch.set_facecolor('black')

for idx, row in pdat.iterrows():    
    wellid = row[COL_WELL]
    i,j = well_dict[wellid]
    curax = ax[i,j]

    curslice = row[COL_SLICE]
    pimg = img[curslice]
    curax.imshow(pimg)
    curax.set_aspect(1)
    curax.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')
    curax.tick_params(
    axis='y',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')
    #scalebar = ScaleBar(0.000001, location=4, color='white',pad=0.1,
    #                    frameon=False,
    #                    font_properties={'size': 5, 'family':'normal'}) # 1 pixel = 0.2 meter
    #scalebar.set_units = 'um'
    #curax.add_artist(scalebar)
#plt.suptitle(met)
for curax in ax.flatten():
    curax.set_facecolor('black')

In [ ]:
def bc_img_process(imcimg, color_dict, sigma=4, div=2):
    imgs = []
    for m, col in color_dict.items():
        img = imcimg.get_img_by_metal(m)
        img = (gaussian_filter(img,sigma)/div)
        img[img >1]=1
        #imgs.append(m*c)
        img=np.stack([img * c for c in col], axis=2)
        imgs.append(img)
    img = np.sum(imgs, axis=0)
    img = img /np.sum(list(color_dict.values()), axis=0).max()
    return img
    
img = bc_img_process(imc_img, color_dict)
plt.imshow(img)

In [ ]:
img = bc_img_process(imc_img, color_dict, sigma=6, div=0.5)
plt.imshow(img)
plt.axis('off')

In [ ]:
def plot_siteimg_as_plate(img, well_slices, well_dict):
    """
    img: an image
    well_slices: a tuple with: (well, image_slice), eg from dat[[COL_WELL, COL_SLICE]].to_records(index=False)
    """
    fig, ax = plt.subplots(nrows=6, ncols=10, subplot_kw={'xticks': [], 'yticks': []},squeeze=True)
    fig.patch.set_facecolor('black')

    for well, sl in well_slices:    
        i,j = well_dict[well]
        curax = ax[i,j]
        pimg = img[sl]
        curax.imshow(pimg)
        curax.set_aspect(1)
        curax.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
        curax.tick_params(
        axis='y',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
        #scalebar = ScaleBar(0.000001, location=4, color='white',pad=0.1,
        #                    frameon=False,
        #                    font_properties={'size': 5, 'family':'normal'}) # 1 pixel = 0.2 meter
        #scalebar.set_units = 'um'
        #curax.add_artist(scalebar)
    #plt.suptitle(met)
    for curax in ax.flatten():
        curax.set_facecolor('black')
    return fig

In [ ]:
def plot_imgs_as_plate(wells_imgs, well_dict, shareax=False, scalebar=False):
    """
    wells_imgs: a list of tuples (well, img)
    """
    
    fig, ax = plt.subplots(nrows=6, ncols=10, subplot_kw={'xticks': [], 'yticks': []},
                           squeeze=True)
    fig.patch.set_facecolor('white')

    for well, img in wells_imgs:    
        i,j = well_dict[well]
        curax = ax[i,j]
        curax.imshow(img)
        curax.set_aspect(1)
        curax.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
        curax.tick_params(
        axis='y',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
        if scalebar:
            scalebar = ScaleBar(0.000001, location=4, color='white',pad=0.1,
                                frameon=False,
                                font_properties={'size': 5, 'family':'normal'}) # 1 pixel = 0.2 meter
            scalebar.set_units = 'um'
            curax.add_artist(scalebar)
    #plt.suptitle(met)
    for curax in ax.flatten():
        curax.set_facecolor('black')
    
    if shareax:
        lims = [max(v) for v in zip(*[(ax.get_xlim()[1], ax.get_ylim()[0]) for ax in fig.axes])]
        for ax in fig.axes:
            xcur = ax.get_xlim()[1]
            ycur = ax.get_ylim()[0]
            dx = (lims[0]-xcur)
            dy = (lims[1]-ycur)
            ax.set_xlim(-dx, xcur+dx)
            ax.set_ylim(ycur+dy, -dy)
    return fig

In [ ]:
fig = plot_siteimg_as_plate(img, pdat[[COL_WELL, COL_SLICE]].values, well_dict)

Plot the  'biggest' section per well

-> let's select the section with the highest nuber of valid barcodes as the 'biggest' section

In [ ]:
from sqlalchemy import func

In [ ]:
dat_maximg = (dat_image
     .groupby(db.conditions.condition_id.key)
     .apply(lambda x: x[db.images.image_id.key][x[db.images.bc_highest_count.key].idxmax()]
           ).rename(db.images.image_id.key)
).reset_index()

In [ ]:
dat_bigbcimgs = (dat_image
 .merge(dat_maximg)
 .merge(dat_cond
       .query(f'{db.conditions.plate_id.key} == 176'))
)

In [ ]:
dat_cond.columns

In [ ]:
COL_IMG = 'img'
COL_IMC = 'imc'

In [ ]:
dat_bigbcimgs[COL_IMC] = (dat_bigbcimgs[db.images.image_id.key]
                          .map(bro.io.imcimg.get_imcimg))

In [ ]:
dat_bigbcimgs[COL_IMG] = (dat_bigbcimgs[COL_IMC]
                 .apply(bc_img_process, color_dict=color_dict, sigma=6, div=0.5))

In [ ]:
plot_imgs_as_plate(dat_bigbcimgs[[COL_WELL, COL_IMG]].values, well_dict, shareax=True)

This looks fairly good - also try once to get the 'biggest' sphere as the sphere
with the most valid objects

In [ ]:
from sqlalchemy import func

In [ ]:
COL_NOBJ = 'nobjects'

In [ ]:
q = (bro.session.query(db.conditions.condition_id, db.images.image_id,
                       func.count(db.objects.object_id).label(COL_NOBJ))
     .join(db.images, db.images.condition_id == db.conditions.condition_id)
     .join(db.objects)
     .join(db.valid_objects)
     .group_by(db.conditions.condition_id, db.images.image_id)
     .filter(db.conditions.plate_id == 176)
)

obj_ct = q.subquery()

In [ ]:
maxobj_ct = (bro.session.query(obj_ct.c.condition_id, func.max(obj_ct.c[COL_NOBJ]).label(COL_NOBJ))
    .group_by(obj_ct.c.condition_id)
    ).subquery('maxct')

In [ ]:
q = (bro.session.query(obj_ct)
         .join(maxobj_ct, obj_ct.c.condition_id == maxobj_ct.c.condition_id)
        .filter(obj_ct.c[COL_NOBJ] == maxobj_ct.c[COL_NOBJ]))

In [ ]:
dat_maxobj = bro.doquery(q).merge(dat_cond)

In [ ]:
dat_maxobj[COL_IMC] = (dat_maxobj[db.images.image_id.key]
                          .map(bro.io.imcimg.get_imcimg))

In [ ]:
dat_maxobj[COL_IMG] = (dat_maxobj[COL_IMC]
                 .apply(bc_img_process, color_dict=color_dict, sigma=6, div=0.5 ))

In [ ]:
fig = plot_imgs_as_plate(dat_maxobj[[COL_WELL, COL_IMG]].values, well_dict, shareax=False)

In [ ]:
fig = plot_imgs_as_plate(dat_maxobj[[COL_WELL, COL_IMG]].values, well_dict, shareax=True)

fig.savefig(C.fol_paper / 'fig1_bcspheres_plate.png')

In [ ]:
## Plot color image in plate layout
p = 99.9

sat = 2

def do_coolorplot(imc):
    img = pct.plots.get_7_color_img(imc, sigma=0.7,
                 metals=['Yb174', # Pan Cadherin
                         'Er167', #GFP
                         'Eu153', #pHistone
                         'Ir191', #Iridium
                         'Er168', #Ki67
                         'Nd145', #pMAPKAPK
                         'Yb172'],# Cleaved Caspase
                alphas=[0.2, #r
                        0.3, #g
                        0.3, #b
                        0.3, #white
                        0.3, # cyan
                        0., #mangenta
                       0.3], saturation=sat,norm_perc=99.99)#yellow
    return img

In [ ]:
dat_maxobj[COL_IMG] = (dat_maxobj[COL_IMC]
                 .apply(do_coolorplot ))

In [ ]:
fig = plot_imgs_as_plate(dat_maxobj[[COL_WELL, COL_IMG]].values, well_dict, shareax=True)

In [ ]:
def get_channels_imgmax(channels, ac):
    q_max = (bro.data.get_measurement_query()
     .with_entities((func.max(db.object_measurements.value)).label(db.object_measurements.value.key),
                db.ref_planes.channel_name,  db.ref_stacks.scale)
     .group_by(db.images.image_id, db.measurements.measurement_id, db.ref_stacks.scale,
              db.ref_planes.channel_name)
     .filter(db.ref_planes.channel_name.in_(channels))
     .filter(db.objects.object_type == 'cell')
     .filter(db.measurement_names.measurement_name == 'MeanIntensity')
     .filter(db.stacks.stack_name == 'FullStackFiltered')
     .filter(db.acquisitions.acquisition_id == ac)
    ).subquery()
    
    q_max2 = (bro.session.query(q_max.c.channel_name,q_max.c.scale,
                             func.avg(q_max.c.value).label(db.object_measurements.value.key),
                   )
        .group_by(q_max.c.channel_name, q_max.c.scale)).subquery()
    
    x =  {chan: val for chan, val in 
          (bro.session.query(q_max2.c.channel_name, q_max2.c.value*q_max2.c.scale))}
    
    
    return x

In [ ]:
def get_all_channels_imgmax(ac):
    q_meas = (bro.data.get_measmeta_query()
     .with_entities(db.measurements.measurement_id,
                db.ref_planes.channel_name,  db.ref_stacks.scale)
     #.group_by(db.images.image_id, db.measurements.measurement_id, db.ref_stacks.scale,
     #         db.ref_planes.channel_name)
     #.filter(db.ref_planes.channel_name.in_(channels))
     .filter(db.measurement_names.measurement_name == 'MeanIntensityComp')
     .filter(db.stacks.stack_name == 'FullStackFiltered')      
             )
              
    q_obj = (bro.data.get_objectmeta_query(valid_images=False) 
     .filter(db.images.acquisition_id == ac)
     .filter(db.objects.object_type == 'cell')
    )
    
    
    ad = bro.io.objmeasurements.get_measurements(q_meas=q_meas, q_obj=q_obj)
    bro.io.objmeasurements.scale_anndata(ad)
    return {c: m for c, m in zip(ad.var[db.ref_planes.channel_name.key], ad.X.max(0))}

In [ ]:
%%time
ad = get_all_channels_imgmax(132)

In [ ]:
ad

This doesnt seem to work!

In [ ]:
metals=['Yb174', # Pan Cadherin
                         'Er167', #GFP
                         'Eu153', #pHistone
                         'Ir193', #Iridium
                         'Er168', #Ki67
                         'Nd145', #pMAPKAPK
                         'Yb172']

In [ ]:
%%time
metals_max =  get_all_channels_imgmax( int(acquisition_id))

-> Quick and dirty: do it just based on 1 acquisition

In [ ]:
def marker_img_process(imcimg, color_dict, div_dict, sigma=0.75,
                      mult=1):
    imgs = []
    for m, col in color_dict.items():
        img = imcimg.get_img_by_metal(m)
        img = (gaussian_filter(img,sigma)/div_dict[m])*mult
        img[img >1]=1
        #imgs.append(m*c)
        img=np.stack([img * c for c in col], axis=2)
        imgs.append(img)
    img = np.sum(imgs, axis=0)
    img = img /np.sum(list(color_dict.values()), axis=0).max()
    return img

In [ ]:
metals_max

In [ ]:
marker_cols = {'Yb174': (1, 0, 0),
 'Er167': (0, 1, 0),
 'Eu153': (0, 0, 1),
 'Er168': (0, 1, 1),
 'Nd145': (1, 0, 1),
 'Yb172': (1, 1, 0)}

metals=['Yb174', # Pan Cadherin
                         'Er167', #GFP
                         'Eu153', #pHistone
                         'Ir193', #Iridium
                         'Er168', #Ki67
                         'Nd145', #pMAPKAPK
                         'Yb172']
img = marker_img_process(imc_img, marker_cols,
                         metals_max, mult=10)

In [ ]:
plt.imshow(img)

In [ ]:
dat_maxobj[COL_IMG] = (dat_maxobj[COL_IMC]
                 .map(lambda x: marker_img_process(x,  marker_cols,
                         metals_max,mult=10))
                      )

In [ ]:
plot_imgs_as_plate(dat_maxobj[[COL_WELL, COL_IMG]].values,
                   well_dict, shareax=True)

In [ ]:
plot_imgs_as_plate(dat_maxobj[[COL_WELL, COL_IMG]].values, well_dict, shareax=True)

## Example plots
That looks not to bad - now find the sphere that should be used as an example sphere and optimize the visualization to show two example 7 color images.

1 of the settings will then be further used to do the slide overview.


Markers:
cellline specific:
- Vimentin - Dy162
- Carbonic anhydrase - Er166
- p-FAK - Nd143
- CD44 - Nd150
- pGSK3 beta - Gd158
- pAKT - Dy163
- Iridium - Ir193


Spatial:
- met

The example image should be from HT29
-> plot all of them

In [ ]:
ref_cond = ['HT-29_c0.5_tp96']

In [ ]:
dat_refimgs = (dat_image
        .query(f'{db.images.acquisition_id.key}=={acquisition_id}')
        .merge(dat_cond)
        .query(f'{db.conditions.condition_name.key} in {ref_cond}')
       )
ref_imgids = [[int(i) for i in dat_refimgs[db.images.image_id.key]][1]]
print(ref_imgids)

Plot Carbonic Anhydrase, CD44, pS6, ki67

In [ ]:
example_mark = ['Yb171', 'Er166', 'Er168', 'Eu153', 'Dy161', 'Nd150', 'Gd158', 'Nd143', 'Yb174','Lu175',
               ]

In [ ]:
imgids = ref_imgids[0]
print(imgids)

In [ ]:
import matplotlib.colors as colors

In [ ]:
C.fol_paper = pathlib.Path('results/figures/')

In [ ]:
for m in example_mark:
    d=bro.plots.heatmask.get_heatmask_data({db.measurements.measurement_name.key: 'MeanIntensityComp',
                                     db.objects.object_type.key: 'cell',
                                     db.ref_planes.channel_name.key: m,
                                     db.stacks.stack_name.key: 'FullStackFiltered'}, image_ids=ref_imgids)
    for imid, dat in d.groupby(db.images.image_id.key):
        dat[db.object_measurements.value.key] =  np.log10(dat[db.object_measurements.value.key]+0.1)
        img = bro.plots.heatmask.assemble_heatmap_image(dat)
        fig, ax = plt.subplots(1,1, figsize=(3,2))
        ax=bro.plots.heatmask.do_heatplot(img,ax=ax)
        scalebar = ScaleBar(0.000001, location=4,color='white',pad=0.1,fixed_value=100,fixed_units='um',
                            frameon=False,
                            font_properties={'size': 10, 'family':'normal'})
        ax.add_artist(scalebar)
        ax.axis('off')
        aximg = ax.images[0]
        #(vmin, vmax) = (aximg.norm.vmin, aximg.norm.vmax)
        #aximg.set_norm(colors.LogNorm(vmin=vmin, vmax=vmax))
        plt.title(f'{bro.helpers.dbhelp.get_target_by_channel(m)} - {m}')
        plt.show()
        fig = ax.get_figure()
        fig.savefig(C.fol_paper / f'fig2_examplehmimg_{m}.pdf' ,transparent=True, dpi=800)

In [ ]:
cur_slices = dat_refimgs.query(f'{db.images.image_id.key} in {ref_imgids}').apply(get_image_slice, axis=1)

In [ ]:
def do_posterplot2(imc_img, chans, alphas, basename, cur_slices):
    img = pct.plots.get_7_color_img(imc_img,
                               metals=chans,
                               alphas=alphas, sigma=0.5,
                                   saturation=sat)

    fig, ax =plt.subplots()
    plt.imshow(img)
    scalebar = ScaleBar(0.000001, location=4,color='white',pad=0.1,
                            frameon=False,
                            font_properties={'size': 10, 'family':'normal'}) # 1 pixel = 0.2 meter
    plt.axis('off')
    ax.add_artist(scalebar)
    #plt.savefig(os.path.join(fol_output, basename+'_overview.png'), dpi=300, bbox_inches='tight', pad_inches=0)

    for cur_slice in cur_slices:
        fig, ax =plt.subplots()
        plt.imshow(img[cur_slice])
        scalebar = ScaleBar(0.000001, location=4,color='white',pad=0.1,
                                frameon=False,
                                font_properties={'size': 10, 'family':'normal'}) # 1 pixel = 0.2 meter
        plt.axis('off')
        ax.add_artist(scalebar)
    #plt.savefig(os.path.join(fol_output, basename+'_zoom.png'), dpi=300, bbox_inches='tight', pad_inches=0)
    #|plt.ion()

In [ ]:
vis_settings = [
    [('Yb171', .5), #Vimenting
    ('Er166', .5), # Carbonic anhydrase
     
     ('Ir193',0.5),
     (0, 0.5),
    ('Er168', 0.7), #Ki67,
     (0,0.5),
    ('Yb174', 0.5) # Parp
    ],
    
    
    [('Nd148', .5), #rb 
    ('Yb176', .5), # Survivin
     ('Tb159', 0.3), #GAPDH
     
     (0, 0.5),
    ('Gd158',0.5),#pr
     (0,0.5),
    ('Eu153', 0.5) # # 
    ],
    
    [('Yb174', .7), #actin 
    ('Gd160', .5), # CD44
     ('La139', 0.7), #trimet
     
     (0, 0.5),
    ('Gd156',0.75),#Cycline B1
     (0,0.5),
    ('Sm152', 0.5) # AMPK
    ],
]
basenames = ['img1', 'img2', 'img3']

In [ ]:
sat=1.5

for i, basename in enumerate(basenames):
    chans, alphas = zip(*vis_settings[i]) 
    basename = basenames[i]
    do_posterplot2(imc_img, chans, alphas, basename, cur_slices)

In [ ]:
%%time
chan_max = get_all_channels_imgmax(int(acquisition_id))

In [ ]:
chan_max

In [ ]:
fol_markimgs = (C.fol_plts / 'marker_imgs')
fol_markimgs.mkdir(exist_ok=True)

In [ ]:
marker_cols = {'Yb174': (1, 0, 0),
 'Er167': (0, 1, 0),
 'Eu153': (0, 0, 1),
 'Er168': (0, 1, 1),
 'Nd145': (1, 0, 1),
 'Yb172': (1, 1, 0)}

for m, title in bro.data.pannel.query(f'(working ==1 )')[['metal', 'goodname']].values:
    marker_cols = {m: (1,1,1)}
    img = marker_img_process(imc_img, marker_cols,
                             chan_max, mult=1)
    plt.figure(figsize=(15,15))
    plt.imshow(img)
    plt.title(title)
    #fig.save(fol_markimgs / '')
    fig, axs = plt.subplots(1, len(cur_slices), figsize=(8,24))
    for ax, sl in zip([axs], cur_slices):
        ax.imshow(img[sl])
        ax.axis('off')

    plt.title(title)
    plt.tight_layout()

plt.close('all')

In [ ]:
marker_cols = {
    'Yb171':  (1,0,0), #Vimenting
    'Er166': (0,1,0), # Carbonic anhydrase
     'Ir193': (0,0,1),
     'Er168':  (1,0,1), #Ki67,
     'Yb174':  (0,1,1) # Pa
}
img = marker_img_process(imc_img, marker_cols,
                         chan_max, mult=0.2)
plt.figure(figsize=(15,15))
plt.imshow(img)

In [ ]:
fig, axs = plt.subplots(1, len(cur_slices), figsize=(8,24))
for ax, sl in zip([axs], cur_slices):
    ax.imshow(img[sl])
    ax.axis('off')

plt.tight_layout()

In [ ]:
def get_all_channels_perc(imc, percentile=99.9):
    perc = {m: np.percentile(imc.get_img_by_metal(m)[:], percentile)
            for m in imc.channel_metals}
    
    
    return perc

In [ ]:
chan_perc = get_all_channels_perc(imc_img, percentile=99.99)

In [ ]:
chan_perc

In [ ]:
marker_cols_n ={
 'Nd150':  ((1,1,0),2),
    'Dy162':  ((1,0,1),1),
      'Dy163':  ((0,1,1),0.7), # pAkt
     'Ir193': ((1,1,1),0.1)
}
chan_perc_upt = {m: chan_perc[m]*mod for m, (c,mod) in marker_cols_n.items()}
marker_cols =  {m: c for m, (c,mod) in marker_cols_n.items()}
img = marker_img_process(imc_img, marker_cols,
                         chan_perc_upt, mult=5)
plt.figure(figsize=(15,15))
plt.imshow(img)
fig, axs = plt.subplots(1, len(cur_slices), figsize=(8,24))
for ax, sl in zip([axs], cur_slices):
    ax.imshow(img[sl])
    ax.axis('off')

plt.tight_layout()

In [ ]:
def plot_labels(marker_cols):
    dist = 0.3
    fig = plt.figure(figsize=(3, len(marker_cols)*dist))
    
    for i, (metal, color) in enumerate(reversed(marker_cols.items())):
        name = bro.data.pannel.query(f'{db.pannel.metal.key} == "{metal}"')['goodname'].iloc[0]
        plt.text(0,i*dist, name, c=color[0],clip_on=True)
    fig.axes[0].set_ylim((0,len(marker_cols)*dist))
    plt.axis('off')
    return fig

In [ ]:
marker_cols

In [ ]:
len(marker_cols)

In [ ]:
dat_maxobj[COL_IMG] = (dat_maxobj[COL_IMC]
                       .map(lambda x: marker_img_process(x, marker_cols,
                         chan_perc_upt, mult=10)))

In [ ]:
fig = plot_imgs_as_plate(dat_maxobj[[COL_WELL, COL_IMG]].values,
                   well_dict, shareax=True)
scalebar = ScaleBar(0.000001, location=4,color='white',pad=0.1,
                                frameon=False,
                                font_properties={'size': 5, 'family':'normal'}) # 1 pixel = 0.2 meter
fig.axes[-1].add_artist(scalebar)
fig.savefig(C.fol_paper / 'overview_imc.pdf')

In [ ]:
plt.close('all')

In [ ]:
fig = plot_labels(marker_cols_n)
fig.savefig(C.fol_paper/ 'overview_imc_labels.pdf')

In [ ]:
fig = plot_imgs_as_plate(dat_maxobj[[COL_WELL, COL_IMG]].values,
                   well_dict, shareax=False)

In [ ]:
marker_cols_list = [
    #{
    #'Yb171':  ((1,0,0),1), # pS6
    #'Eu153': ((0,0,0),1), # pHH3
    # 'Ir193': ((0.1,0.1,0.1),0.9), # 
    # 'Er168':  ((0,0,1),0.8), #Ki67,
    # 'Dy161':  ((0,1,0),1) #  Cyclin B1
#},
{
    'Yb171':  ((1,0,1),1), # pS6
   # 'Eu153': ((0,0,0),1), # pHH3
    # 'Ir193': ((0.1,0.1,0.1),0.9), # 
     'Er168':  ((0,1,1),1), #Ki67,
     'Dy161':  ((1,1,0),1) #  Cyclin B1
},
    {
    'Dy163':  ((1,0,1),0.7), # pAkt
    # 'Ir193': ((0.1,0.1,0.1),0.9), # 
     'Eu153':  ((0,1,1),1), #pHH3,
     'Er166':  ((1,1,0),2) #  CArbonic anhydrase
},
    {
    'Lu175':  ((1,0,1),1), # Prb
    # 'Ir193': ((0.1,0.1,0.1),0.9), # 
     'Nd150':  ((0,1,1),3), # CD44
     'La139':  ((1,1,0),1) #  metHH3
},
        {
    'Gd155':  ((1,0,1),1), # pHer2
    # 'Ir193': ((0.1,0.1,0.1),0.9), # 
     'Nd143':  ((0,1,1),1), # FAK
     'Nd145':  ((1,1,0),1) #  MAPKAPK
},
]
for imnr, marker_cols in enumerate(marker_cols_list):
    chan_perc_upt = {m: chan_perc[m]*mod for m, (c,mod) in marker_cols.items()}
    marks = marker_cols
    marker_cols =  {m: c for m, (c,mod) in marker_cols.items()}
    img = marker_img_process(imc_img, marker_cols,
                             chan_perc_upt, mult=10)
    fig = plt.figure(figsize=(2.3,2.3), dpi=1200)
    plt.imshow(img)
    scalebar = ScaleBar(0.000001, location=4,color='white',pad=0.1,
                            frameon=False,
                            font_properties={'size': 10, 'family':'normal'}) # 1 pixel = 0.2 meter
    ax = plt.gca()
    ax.add_artist(scalebar)
    plt.axis('off')
    plt.gcf()
    fig.savefig(C.fol_paper / f'exampleimgs2_{imnr}_big.pdf')
    plot_labels(marks)
    plt.gcf().savefig(C.fol_paper / f'exampleimgs2_{imnr}_label.pdf', dpi=1200)
    fig, axs = plt.subplots(1, len(cur_slices), figsize=(2,2), dpi=1200)
    if len(cur_slices) == 1:
        axs = [axs]
    for ax, sl in zip(axs, cur_slices):
        ax.imshow(img[sl])
        ax.axis('off')
        scalebar = ScaleBar(0.000001, location=4,color='white',pad=0.1,
                                frameon=False,
                                font_properties={'size': 10, 'family':'normal'}) # 1 pixel = 0.2 meter
        plt.axis('off')
        ax.add_artist(scalebar)
    plt.tight_layout()
    plt.show()
    fig.savefig(C.fol_paper / f'exampleimgs2_{imnr}_HT29.pdf')
plt.close('all')

In [ ]:
marker_cols_list = [
    #{
    #'Yb171':  ((1,0,0),1), # pS6
    #'Eu153': ((0,0,0),1), # pHH3
    # 'Ir193': ((0.1,0.1,0.1),0.9), # 
    # 'Er168':  ((0,0,1),0.8), #Ki67,
    # 'Dy161':  ((0,1,0),1) #  Cyclin B1
#},
{
    'Yb171':  ((1,0,0),1), # pS6
         'Er166':  ((0,1,0),10), #  CArbonic anhydrase
     'Ir193': ((0.,0.,1),0.1), # 
   # 'Eu153': ((0,0,0),1), # pHH3

     'Er168':  ((0,1,1),1), #Ki67,
    'Yb174':  ((1,1,0),1) #  parp
},
    {
    'Lu175':  ((1,0,0),0.5), # Prb
         'Nd150':  ((0,1,0),20), # CD44
      'La139':  ((0,0,1),2), #  metHH3

   'Nd143':  ((0,1,1),1), # fak   
    'Eu153':  ((1,1,0),1), #pHH3,
   # 'Dy163':  ((1,0,1),0.7), # pAkt
    # 'Ir193': ((0.1,0.1,0.1),0.9), # 


},
    {
           'Dy161':  ((1,0,0),1.5), #  Cyclin B1
            'Nd145':  ((0,1,0),2), # MAPKAPK 
        'Dy163':  ((0,0,1),2), # pAkt
    # 'Ir193': ((0.1,0.1,0.1),0.9), # 

           'Dy162':  ((0,1,1),3), # vimentin  

      'Sm152':  ((1,1,0),4), #  AMPK
     
        
},
]
for imnr, marker_cols in enumerate(marker_cols_list):
    chan_perc_upt = {m: chan_perc[m]*mod for m, (c,mod) in marker_cols.items()}
    marks = marker_cols
    marker_cols =  {m: c for m, (c,mod) in marker_cols.items()}
    img = marker_img_process(imc_img, marker_cols,
                             chan_perc_upt, mult=40)
    fig = plt.figure(figsize=(2.3,2.3), dpi=1200)
    plt.imshow(img)
    scalebar = ScaleBar(0.000001, location=4,color='white',pad=0.1,
                            frameon=False,
                            font_properties={'size': 10, 'family':'normal'}) # 1 pixel = 0.2 meter
    ax = plt.gca()
    ax.add_artist(scalebar)
    plt.axis('off')
    plt.gcf()
    fig.savefig(C.fol_paper / f'exampleimgs_{imnr}_big.pdf')
    plot_labels(marks)
    plt.gcf().savefig(C.fol_paper / f'exampleimgs_{imnr}_label.pdf', dpi=1200)
    fig, axs = plt.subplots(1, len(cur_slices), figsize=(2,2), dpi=1200)
    if len(cur_slices) == 1:
        axs = [axs]
    for ax, sl in zip(axs, cur_slices):
        ax.imshow(img[sl])
        ax.axis('off')
        scalebar = ScaleBar(0.000001, location=4,color='white',pad=0.1,
                                frameon=False,
                                font_properties={'size': 10, 'family':'normal'}) # 1 pixel = 0.2 meter
        plt.axis('off')
        ax.add_artist(scalebar)
    plt.tight_layout()
    plt.show()
    fig.savefig(C.fol_paper / f'exampleimgs_{imnr}_HT29.pdf')
plt.close('all')

In [ ]:

for marker_cols in marker_cols_list:
    plot_labels(marker_cols)

-> Looks good, now I just need to find the best combinations of markers

In [ ]:
dat_image.query(f'{db.images.image_id.key} == 194')